In [40]:
import json
import numpy as np
import pandas as pd
from pathlib import Path
from pose_format import Pose
from pose_format.numpy import NumPyPoseBody
from pose_format.pose_header import PoseHeader, PoseHeaderComponent, PoseHeaderDimensions

## 📥 Read the .pose file first

In [41]:
def load_pose_file(pose_path: str) -> Pose:
    """
    Reads a .pose file and returns a Pose object.
    """
    with open(pose_path, 'rb') as f:
        pose = Pose.read(f.read())
    return pose


In [42]:
path_pose = '../data/pose_files/example.pose'
pose = load_pose_file(path_pose)

## 1️⃣ Save as .pose (original format)

In [43]:
save_path = "../output/convert_pose_formats"

In [44]:
def save_as_pose(pose: Pose, output_path: str):
    """
    Saves a Pose object to a .pose file.
    """
    with open(output_path, 'wb') as f:
        pose.write(f)
    print(f"✅ Saved to: {output_path}")


save_as_pose(pose, f"{save_path}/output.pose")

✅ Saved to: ../output/convert_pose_formats/output.pose


## 2️⃣ Convert to JSON

In [48]:

def build_pose_from_arrays(
    data: np.ndarray,
    confidence: np.ndarray,
    fps: float,
    header: PoseHeader,
    mask: np.ndarray = None
 ) -> Pose:
    """
    Build a Pose object from raw arrays.
    IMPORTANT: Uses NumPyPoseBody to ensure proper write() functionality.
    """
    if mask is not None:
        data = np.ma.MaskedArray(data, mask=mask)
    # Use NumPyPoseBody instead of generic PoseBody for write support
    body = NumPyPoseBody(fps, data, confidence)
    return Pose(header, body)




In [49]:
def pose_to_json_compact(pose: Pose, output_path: str):
    """
    Convert Pose to compact JSON (includes full header for standalone conversion).
    """
    data = pose.body.data.filled(0).tolist()
    confidence = pose.body.confidence.tolist()
    
    # Include full header for independent reconstruction
    result = {
        "d": data,
        "c": confidence,
        "f": float(pose.body.fps),
        "h": {
            "v": float(pose.header.version),
            "w": int(pose.header.dimensions.width),
            "h": int(pose.header.dimensions.height),
            "d": int(pose.header.dimensions.depth),
            "comps": [
                {
                    "n": comp.name,
                    "pts": comp.points,
                    "fmt": comp.format,
                    "limbs": [[int(x) for x in limb] if hasattr(limb, '__iter__') else int(limb) for limb in comp.limbs],
                    "colors": [[int(x) for x in c] if hasattr(c, '__iter__') else int(c) for c in getattr(comp, "colors", [])] if hasattr(comp, "colors") else None
                }
                for comp in pose.header.components
            ]
        }
    }
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, separators=(',', ':'))
    
    print(f"✅ Saved (compact) to: {output_path}")

In [50]:
def json_compact_to_pose(
    json_path: str,
    output_path: str
 ) -> Pose:
    """
    Convert compact JSON back to a .pose file (fully standalone).
    """
    with open(json_path, "r", encoding="utf-8") as f:
        compact = json.load(f)

    data = np.array(compact["d"], dtype=np.float32)
    confidence = np.array(compact["c"], dtype=np.float32)
    fps = float(compact["f"])
    
    # Reconstruct header from embedded data
    header_data = compact.get("h")
    if not header_data:
        raise ValueError("No header found in compact JSON file. File may be corrupted or incompatible.")
        
    # Build header from embedded info
    dimensions = PoseHeaderDimensions(
        width=header_data["w"],
        height=header_data["h"],
        depth=header_data.get("d", 0)
    )
    components = []
    for comp in header_data.get("comps", []):
        limbs = [tuple(limb) for limb in comp.get("limbs", [])]
        colors = comp.get("colors")
        if not colors:
            colors = [(255, 255, 255)] * len(limbs)
        components.append(
            PoseHeaderComponent(
                name=comp["n"],
                points=comp["pts"],
                limbs=limbs,
                colors=colors,
                point_format=comp.get("fmt", "XYZC")
            )
        )
    header = PoseHeader(
        version=header_data.get("v", 0.1),
        dimensions=dimensions,
        components=components
    )
    
    pose = build_pose_from_arrays(data, confidence, fps, header)
    
    # Save to file
    with open(output_path, 'wb') as f:
        pose.write(f)
    
    return pose


In [51]:
# Example: compact JSON -> .pose (now standalone!)
pose_to_json_compact(pose, f"{save_path}/output.compact.json")


✅ Saved (compact) to: ../output/convert_pose_formats/output.compact.json


In [52]:
restored_pose_compact = json_compact_to_pose(f"{save_path}/output.compact.json", f"{save_path}/restored_from_compact.pose")

## 3️⃣ Convert to NumPy (.npz)

In [53]:
def pose_to_npz(pose: Pose, output_path: str, compressed: bool = True):
    """
    Convert Pose to NumPy (.npz) with full header information for standalone conversion.
    
    Parameters:
    -----------
    pose : Pose
        Pose object.
    output_path : str
        Output file path.
    compressed : bool
        Use compression (smaller size but slower read/write).
    """
    # Extract data
    data = np.array(pose.body.data.filled(0), dtype=np.float32)
    confidence = np.array(pose.body.confidence, dtype=np.float32)
    mask = np.array(pose.body.data.mask, dtype=bool)
    
    # Serialize components info as JSON string
    components_data = []
    for comp in pose.header.components:
        components_data.append({
            "name": comp.name,
            "points": comp.points,
            "format": comp.format,
            "limbs": [[int(x) for x in limb] if hasattr(limb, '__iter__') else int(limb) for limb in comp.limbs],
            "colors": [[int(x) for x in c] if hasattr(c, '__iter__') else int(c) for c in getattr(comp, "colors", [])] if hasattr(comp, "colors") else None
        })
    components_json = json.dumps(components_data)
    
    # Extra metadata
    metadata = {
        'fps': np.array([pose.body.fps]),
        'width': np.array([pose.header.dimensions.width]),
        'height': np.array([pose.header.dimensions.height]),
        'depth': np.array([pose.header.dimensions.depth]),
        'version': np.array([pose.header.version]),
        'components': np.array([components_json], dtype=object)
    }
    
    # Save
    save_func = np.savez_compressed if compressed else np.savez
    save_func(
        output_path,
        data=data,
        confidence=confidence,
        mask=mask,
        **metadata
    )
    
    print(f"✅ Saved to: {output_path}")
    print(f"   📦 File size: {Path(output_path).stat().st_size / 1024:.2f} KB")



In [54]:
pose_to_npz(pose, f"{save_path}/output.npz", compressed=True)

✅ Saved to: ../output/convert_pose_formats/output.npz
   📦 File size: 306.53 KB


In [55]:
def npz_to_pose(npz_path: str, output_path: str) -> Pose:
    """
    Convert NPZ back to a .pose file (fully standalone).
    """
    loaded = np.load(npz_path, allow_pickle=True)
    
    data = loaded['data'].astype(np.float32)
    confidence = loaded['confidence'].astype(np.float32)
    mask = loaded['mask'] if 'mask' in loaded.files else None
    fps = float(loaded['fps'][0]) if 'fps' in loaded.files else 30.0
    
    # Load header from NPZ
    if 'components' not in loaded.files:
        raise ValueError("No header found in NPZ file. File may be corrupted or incompatible.")
        
    # Reconstruct header from saved components
    width = int(loaded['width'][0]) if 'width' in loaded.files else data.shape[2]
    height = int(loaded['height'][0]) if 'height' in loaded.files else data.shape[1]
    depth = int(loaded['depth'][0]) if 'depth' in loaded.files else 0
    version = float(loaded['version'][0]) if 'version' in loaded.files else 0.1
    
    dimensions = PoseHeaderDimensions(width, height, depth)
    
    # Deserialize components
    components_json = str(loaded['components'][0])
    components_data = json.loads(components_json)
    
    components = []
    for comp_data in components_data:
        limbs = [tuple(limb) for limb in comp_data.get("limbs", [])]
        colors = comp_data.get("colors")
        if not colors:
            colors = [(255, 255, 255)] * len(limbs)
        components.append(
            PoseHeaderComponent(
                name=comp_data["name"],
                points=comp_data["points"],
                limbs=limbs,
                colors=colors,
                point_format=comp_data.get("format", "XYZC")
            )
        )
    
    header = PoseHeader(
        version=version,
        dimensions=dimensions,
        components=components
    )
    
    pose = build_pose_from_arrays(data, confidence, fps, header, mask=mask)
    
    # Save to file
    with open(output_path, 'wb') as f:
        pose.write(f)
    
    return pose



In [56]:
# Example: NPZ -> .pose (now standalone!)
restored_pose_npz = npz_to_pose(f"{save_path}/output.npz", f"{save_path}/restored_from_npz.pose")

## 6️⃣ Comprehensive conversion function

In [57]:
# Test Standalone Conversions (No reference_pose needed!)
print("🧪 Testing Standalone Conversions...\n")

# 1. Test Compact JSON
print("1️⃣ Testing Compact JSON...")
pose_to_json_compact(pose, f"{save_path}/test.compact.json")
restored_compact = json_compact_to_pose(f"{save_path}/test.compact.json", f"{save_path}/test_compact_restored.pose")
print(f"   ✅ Original shape: {pose.body.data.shape}")
print(f"   ✅ Restored shape: {restored_compact.body.data.shape}")
print("✅ Compact JSON: PASSED\n")

# 2. Test NPZ
print("2️⃣ Testing NPZ...")
pose_to_npz(pose, f"{save_path}/test.npz", compressed=True)
restored_npz = npz_to_pose(f"{save_path}/test.npz", f"{save_path}/test_npz_restored.pose")
print(f"   ✅ Original shape: {pose.body.data.shape}")
print(f"   ✅ Restored shape: {restored_npz.body.data.shape}")
print("✅ NPZ: PASSED\n")

print("=" * 60)
print("🎉 ALL CONVERSIONS ARE NOW FULLY STANDALONE!")
print("=" * 60)
print("No reference_pose parameter needed anywhere! ✨")
print("Each format saves complete header information independently.")

🧪 Testing Standalone Conversions...

1️⃣ Testing Compact JSON...
✅ Saved (compact) to: ../output/convert_pose_formats/test.compact.json
   ✅ Original shape: (133, 1, 203, 3)
   ✅ Restored shape: (133, 1, 203, 3)
✅ Compact JSON: PASSED

2️⃣ Testing NPZ...
✅ Saved to: ../output/convert_pose_formats/test.npz
   📦 File size: 306.53 KB
   ✅ Original shape: (133, 1, 203, 3)
   ✅ Restored shape: (133, 1, 203, 3)
✅ NPZ: PASSED

🎉 ALL CONVERSIONS ARE NOW FULLY STANDALONE!
No reference_pose parameter needed anywhere! ✨
Each format saves complete header information independently.


In [ ]:
def convert_pose(pose_path: str, output_format: str, output_path: str = None):
    """
    Comprehensive conversion function.
    
    Parameters:
    -----------
    pose_path : str
        Path to the .pose file.
    output_format : str
        Desired format: 'json', 'npz'.
    output_path : str, optional
        Output file path (auto-generated if not provided).
    """
    # Load file
    pose = load_pose_file(pose_path)
    
    # Build output filename
    base_path = Path(pose_path).stem
    
    format_map = {
        'json': ('.json', pose_to_json_compact),
        'npz': ('.npz', pose_to_npz)
    }
    
    if output_format not in format_map:
        raise ValueError(
            f"Unsupported format: {output_format}. Supported formats: {list(format_map.keys())}"
        )
    
    ext, converter = format_map[output_format]
    
    if output_path is None:
        output_path = f"{base_path}{ext}"
    
    return converter(pose, output_path)

# Usage examples:
# convert_pose("input.pose", "json")
# convert_pose("input.pose", "npz")

In [58]:
# Test convert_pose function
print("🧪 Testing convert_pose function...")

# Test JSON conversion
convert_pose(path_pose, "json", f"{save_path}/test_convert.json")

# Test NPZ conversion
convert_pose(path_pose, "npz", f"{save_path}/test_convert.npz")

print("✅ All convert_pose tests passed!")

🧪 Testing convert_pose function...
✅ Saved (compact) to: ../output/convert_pose_formats/test_convert.json
✅ Saved to: ../output/convert_pose_formats/test_convert.npz
   📦 File size: 306.53 KB
✅ All convert_pose tests passed!


## 7️⃣ Batch conversion

In [ ]:
from pathlib import Path
from tqdm import tqdm

def batch_convert(input_dir: str, output_format: str, output_dir: str = None):
    """
    Convert a folder of .pose files.
    """
    input_path = Path(input_dir)
    pose_files = list(input_path.glob("*.pose"))
    
    if not pose_files:
        print("⚠️ No .pose files found")
        return
    
    print(f"📁 Found {len(pose_files)} files")
    
    # Create output folder
    if output_dir is None:
        output_dir = input_path / f"converted_{output_format}"
    else:
        output_dir = Path(output_dir)
    
    output_dir.mkdir(exist_ok=True)
    
    # Convert files
    for pose_file in tqdm(pose_files, desc="Converting"):
        try:
            ext = {'json': '.json', 'npz': '.npz'}[output_format]
            output_path = output_dir / f"{pose_file.stem}{ext}"
            convert_pose(str(pose_file), output_format, str(output_path))
        except Exception as e:
            print(f"❌ Error in {pose_file.name}: {e}")
    
    print(f"\n✅ Converted files saved to: {output_dir}")

# batch_convert("pose_files_folder", "json")
# batch_convert("pose_files_folder", "npz")

## 8️⃣ Compare file sizes

In [59]:
def compare_file_sizes(pose_path: str):
    """
    Compare file sizes across different formats - JSON and NPZ only.
    """
    import tempfile
    import os
    
    pose = load_pose_file(pose_path)
    original_size = Path(pose_path).stat().st_size
    
    results = {'pose (original)': original_size}
    
    with tempfile.TemporaryDirectory() as tmpdir:
        # JSON (compact)
        json_path = os.path.join(tmpdir, "test.json")
        pose_to_json_compact(pose, json_path)
        results['json (compact)'] = Path(json_path).stat().st_size
        
        # NPZ (compressed)
        npz_path = os.path.join(tmpdir, "test.npz")
        pose_to_npz(pose, npz_path, compressed=True)
        results['npz (compressed)'] = Path(npz_path).stat().st_size
    
    # Show results
    print("\n" + "=" * 50)
    print("📊 File size comparison")
    print("=" * 50)
    
    for fmt, size in sorted(results.items(), key=lambda x: x[1]):
        ratio = size / original_size * 100
        print(f"{fmt:25} {size/1024:10.2f} KB  ({ratio:6.1f}%)")

In [61]:
compare_file_sizes(f"{save_path}/restored_from_npz.pose")

✅ Saved (compact) to: C:\Users\micrk\AppData\Local\Temp\tmpw7l44yft\test.json
✅ Saved to: C:\Users\micrk\AppData\Local\Temp\tmpw7l44yft\test.npz
   📦 File size: 306.53 KB

📊 File size comparison
npz (compressed)              306.53 KB  (  72.1%)
pose (original)               424.93 KB  ( 100.0%)
json (compact)               1735.05 KB  ( 408.3%)


## 📚 Format summary

| Format | Pros | Cons | Best use |
|--------|------|------|----------|
| `.pose` | Native library format, compressed | Library-specific | Storage and library workflows |
| `.json` | Human-readable, web-friendly | Large size | Web, debugging |
| `.npz` | Efficient for NumPy, compressed | Requires NumPy | Scientific analysis |